# Stock price change (and other features) on Earnings Dates

In [218]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime


from seaborn import set_style
set_style("whitegrid")

import os

from bs4 import BeautifulSoup

In [219]:
def extract_earnings_call_info(content):
    # Updated regex pattern
    ticker_pattern = re.compile(r'\(NASDAQ:(.*?)\)')
    # Non-capturing groups for date alternation, separate capturing groups for date and time
    date_time_pattern = re.compile(r'(?:\b\d{1,2} [A-Za-z]+|\b[A-Za-z]+ \d{1,2}), \d{4}\s+(\d{1,2}[:.]\d{2} [AP]M)\s*(ET|PT)')
    
    ticker_search = ticker_pattern.search(content)
    ticker = ticker_search.group(1) if ticker_search else "Ticker Symbol Not Found"
    
    date_time_search = date_time_pattern.search(content)
    if date_time_search:
        date_time = date_time_search.group(0)
    else:
        date_time = "Date and Time Not Found"
    
    return ticker, date_time

base_path = r"C:\Users\spatan5\Desktop\EarningsNLP"

for company_folder in os.listdir(base_path):
    company_path = os.path.join(base_path, company_folder)
    if os.path.isdir(company_path):
        for file_name in os.listdir(company_path):
            file_path = os.path.join(company_path, file_name)
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as file:
                    content = file.read()
                    soup = BeautifulSoup(content, 'html.parser')
                    text_content = soup.get_text()
                    ticker, date_time = extract_earnings_call_info(text_content)
                    print(f"File: {file_name}, Ticker: {ticker}, Date and Time: {date_time}")


File: config, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: description, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: FETCH_HEAD, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: HEAD, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found


C:\Users\spatan5\AppData\Local\Temp\ipykernel_9212\1373160396.py:28: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, 'html.parser')
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


File: index, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: packed-refs, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: scraping_guide-checkpoint.ipynb, Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
File: Alphabet (GOOG) Q1 2016 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: April 21, 2016 4:30 PM ET
File: Alphabet (GOOG) Q1 2017 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: April 27, 2017 4:30 PM ET
File: Alphabet (GOOG) Q2 2016 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: July 28, 2016 4:30 PM ET
File: Alphabet (GOOG) Q3 2015 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: October 22, 2015 4:30 PM ET
File: Alphabet (GOOG) Q3 2016 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: October 27, 2016 4:30 PM ET
File: Alphabet (GOOG) Q3 2017 Results - Earnings Call Transcript, Ticker: GOOG, Date and Time: October 26

In [220]:
## Load Coca-Cola Data
#file_path = '/Users/tejaswitripathi/Documents/GitHub/Project_Spring_2024/Data/Coca-Cola Company'
with open('C:/Users/spatan5/Desktop/EarningsNLP/Microsoft/Microsoft (MSFT) Q1 2016 Results - Earnings Call Transcript', 'r') as file:
    cola = file.read()

In [221]:
soup = BeautifulSoup(cola, 'html.parser')

In [222]:
text_content = soup.get_text()

In [223]:
print(extract_earnings_call_info(text_content))

('MSFT', 'October 22, 2015 5:30 PM ET')


In [224]:
import yfinance as yf

In [227]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def adjust_for_weekend(target_date):
    """
    Adjusts a date to ensure it falls on a weekday, shifting to the previous Friday if it falls on a weekend.
    """
    weekday = target_date.weekday()
    if weekday == 5:  # Saturday
        return target_date - timedelta(days=1)
    elif weekday == 6:  # Sunday
        return target_date - timedelta(days=2)
    return target_date

def get_stock_prices(ticker, earnings_datetime_str):
    if ticker == "Ticker Symbol Not Found" or earnings_datetime_str == "Date and Time Not Found":
        return "Data not available due to missing information", "Data not available due to missing information"
    
    try:
        earnings_datetime = datetime.strptime(earnings_datetime_str, "%B %d, %Y %I:%M %p ET")
    except ValueError:
        return "Data not available due to incorrect date/time format", "Data not available due to incorrect date/time format"
    
    if earnings_datetime.hour >= 16:  # After market close
        start_date = earnings_datetime + timedelta(days=1)
        end_date = earnings_datetime - timedelta(days=1)
    else:  # Before market close
        start_date = earnings_datetime
        end_date = earnings_datetime - timedelta(days=2)
    
    # Adjust for weekends
    start_date = adjust_for_weekend(start_date)
    end_date = adjust_for_weekend(end_date)

    # Try to fetch day_after_high up to 4 times
    attempts = 0
    day_after_high = "Data not available after multiple attempts"
    while attempts < 4:
        start_date_str = start_date.strftime('%Y-%m-%d')
        try:
            data_after = yf.download(ticker, start=start_date_str, end=(start_date + timedelta(days=1)).strftime('%Y-%m-%d'))
            if not data_after.empty:
                day_after_high = data_after['High'].max()
                break
        except Exception:
            break
        start_date += timedelta(days=1)  # Move to next day if data is not available
        attempts += 1

    # Try to fetch day_before_avg up to 4 times
    attempts = 0
    day_before_avg = "Data not available after multiple attempts"
    while attempts < 4:
        end_date_str = end_date.strftime('%Y-%m-%d')
        try:
            data_before = yf.download(ticker, start=(end_date - timedelta(days=1)).strftime('%Y-%m-%d'), end=end_date_str)
            if not data_before.empty:
                day_before_avg = (data_before['High'] + data_before['Low']).mean() / 2
                break
        except Exception:
            break
        end_date -= timedelta(days=1)  # Move back one day if data is not available
        attempts += 1

    return day_after_high, day_before_avg


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NVDA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-02-19 -> 2024-02-20)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NVDA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-02-18 -> 2024-02-19)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NVDA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-02-17 -> 2024-02-18)')
[*********************100%%**********************]  1 of 1 completed


AttributeError: 'float' object has no attribute 'round'

In [228]:
base_path = r"C:\Users\spatan5\Desktop\EarningsNLP"

for company_folder in os.listdir(base_path):
    company_path = os.path.join(base_path, company_folder)
    if os.path.isdir(company_path):
        for file_name in os.listdir(company_path):
            file_path = os.path.join(company_path, file_name)
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as file:
                    content = file.read()
                    soup = BeautifulSoup(content, 'html.parser')
                    text_content = soup.get_text()
                    ticker, date_time = extract_earnings_call_info(text_content)
                    print(f"Ticker: {ticker}, Date and Time: {date_time}")
                    day_after_high, day_before_avg = get_stock_prices(ticker, date_time)
                    if isinstance(day_after_high, float):
                        day_after_high = round(day_after_high, 2)
                    else:
                        day_after_high = day_after_high  # Or handle the string message as needed
                    if isinstance(day_before_avg, float):
                        day_before_avg = round(day_before_avg, 2)
                    else:
                        day_before_avg = day_before_avg
                    print(f"Highest price on the day after earnings release: {day_after_high}")
                    print(f"Average price on the day before earnings release: {day_before_avg}")
                    if isinstance(day_after_high, float) and isinstance(day_before_avg, float):
                        print(f"Percentage change on earnings day: {round((float(day_after_high) - float(day_before_avg)) * 100 / float(day_before_avg), 2)}%")

Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings rel

C:\Users\spatan5\AppData\Local\Temp\ipykernel_9212\614123346.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, 'html.parser')
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Date and Time Not Found
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: GOOG, Date and Time: April 21, 2016 4:30 PM ET


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Highest price on the day after earnings release: 36.81
Average price on the day before earnings release: 37.98
Percentage change on earnings day: -3.08%
Ticker: GOOG, Date and Time: April 27, 2017 4:30 PM ET


...

Highest price on the day after earnings release: 58.12
Average price on the day before earnings release: 52.59
Percentage change on earnings day: 10.52%
Ticker: TXN, Date and Time: January 27, 2016 5:30 PM ET


[*********************100%%**********************]  1 of 1 completed


Highest price on the day after earnings release: 52.0
Average price on the day before earnings release: 50.5
Percentage change on earnings day: 2.97%
Ticker: Ticker Symbol Not Found, Date and Time: August 8, 2019 5:00 PM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: August 6, 2020 4:30 PM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: February 6, 2020 4:30 PM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: Novem

C:\Users\spatan5\AppData\Local\Temp\ipykernel_9212\614123346.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, 'html.parser')


Ticker: Ticker Symbol Not Found, Date and Time: July 20, 2017 8:45 AM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: October 22, 2015 8:45 AM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: January 20, 2016 8:45 AM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before earnings release: Data not available due to missing information
Ticker: Ticker Symbol Not Found, Date and Time: January 19, 2017 8:45 AM ET
Highest price on the day after earnings release: Data not available due to missing information
Average price on the day before